In [1]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import warnings
warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesse
 
import shap
from dataset import engine, RFM, Churn, Engagement, RFM_engage, RFM_churn
from models import CLFSwitcher, Transform, Pipe, parameters
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'data'

In [ ]:
rfm = RFM(engine)

engage = Engagement(engine)

churn = Churn(engine)
# churn = churn.get_dataset()[['customer_id', 'churn']]

In [ ]:
churn = RFM_churn(rfm, churn)

In [ ]:
# engage = RFM_engage(rfm, engage)

In [ ]:
data = churn
X = data.get_X()
y = data.get_y()
ct = Transform(data)
X, y = ct.get_Xy()
# ct.inverse_transform(pd.concat([X, y], axis=1))
# y

In [ ]:
pipeline = Pipe(ct).get_pipeline()
def train(X, y, pipeline, parameters):
        grid_search = GridSearchCV(pipeline, parameters, cv=2, n_jobs=12, return_train_score=True, verbose=2)
        grid_search.fit(X, y)
        return grid_search, grid_search.best_estimator_[-1]
_, best_estimator = train(X, y, pipeline, parameters)

In [ ]:
best_estimator.explain(data, ct)


In [ ]:
shap_df = best_estimator.get_shap(X_col='yearly_income', y_col='churn', y_val='yes')
shap_df

In [ ]:
plt.scatter(shap_df.iloc[:, -1], shap_df.iloc[:,0])